# Step-by-step implementation
Let’s see how this works in practice with the provided code. We’ll break down the code step-by-step, explaining its functionality and how it implements CrossEncoder Reranking for enhanced information retrieval:
1. Import necessary modules
2. Set up the OpenAI API key
3. Load and split documents
4. Initial retrieval
5. Set up Cross Encoder reranking
6. Rerank with Cross Encoder







## 1. Import necessary modules

In [0]:
import os
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

## 2. Set up the OpenAI API key

In [0]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] = ""  # Add your OpenAI API key
if OPENAI_API_KEY == "":
    raise ValueError("Please set the OPENAI_API_KEY environment variable")

## 3. Load and split documents

In [0]:
loaders = [
    TextLoader("blog.langchain.dev_announcing-langsmith_.txt"),
    TextLoader("blog.langchain.dev_automating-web-research_.txt"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=400, chunk_overlap=60)
splits = text_splitter.split_documents(docs)

## 4. Initial retrieval

In [0]:
# Index
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [0]:
query = "What is LangSmith?"
docs = retriever.invoke(query)
print(docs)

## 5. Set up Cross Encoder reranking

In [0]:
# Replace 'your_huggingface_token_here' with your actual Hugging Face token before executing this cell
!huggingface-cli login --token your_huggingface_token_here

In [0]:
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")

In [0]:
compressor = CrossEncoderReranker(model=model, top_n=3)

In [0]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

## 6. Reranking with CrossEncoder

In [0]:
compressed_docs = compression_retriever.invoke("What is LangSmith?")
print(compressed_docs)